# 6. Route Details

## Observations

- `eta` information is available from this API.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.route_details

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

import json
import time
from tqdm import tqdm

import requests
import pandas as pd
pd.set_option('display.max_columns', None)

from traffic_data_bengaluru.utils import *
from traffic_data_bengaluru.bmtc.apis.vehicles import get_vehicles

In [ ]:
#| export

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false

data_directory = get_data_directory() / "bmtc"

In [ ]:
# | export

def fetch_route_details(route_id: int, sleep_duration: float = 0.1):
    """Fetch route details for a given route ID from the BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchByRouteDetails_v4"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "deviceType": "WEB",
        'lan': 'en'
    }

    payload = json.dumps({"routeid": int(route_id)})
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        print("Response text:", getattr(e.response, "text", None))
        return None

In [ ]:
#| hide
#| eval: false

sample_route_id = 2116
sample_route_details = fetch_route_details(route_id = sample_route_id)
sample_route_details

{'up': {'data': [{'routeid': 3796,
    'stationid': 20921,
    'stationname': 'Kempegowda Bus Station',
    'from': 'Kempegowda Bus Station',
    'to': 'Uttarahalli Bus Stand',
    'routeno': '210-N',
    'distance_on_station': 0,
    'centerlat': 12.97751,
    'centerlong': 77.57141,
    'responsecode': 200,
    'isnotify': 0,
    'vehicleDetails': [{'vehicleid': 3441,
      'vehiclenumber': 'KA53F0268',
      'servicetypeid': 72,
      'servicetype': 'Non AC/Ordinary',
      'centerlat': 12.909246,
      'centerlong': 77.536713,
      'eta': '',
      'sch_arrivaltime': '15:20',
      'sch_departuretime': '15:20',
      'actual_arrivaltime': '16:29',
      'actual_departuretime': '16:29',
      'sch_tripstarttime': '15:20',
      'sch_tripendtime': '15:20',
      'lastlocationid': 0,
      'currentlocationid': 35088,
      'nextlocationid': 0,
      'currentstop': None,
      'nextstop': None,
      'laststop': None,
      'stopCoveredStatus': 1,
      'heading': 0,
      'lastrefres

In [ ]:
# | export

def extract_route_details(route_details):
    details = []
    directions = ['up', 'down']
    for direction in directions:
        stations = route_details[direction]['data']
        for station in stations:
            for vehicle in station['vehicleDetails']:
                detail = {**vehicle, **{k: v for k, v in station.items() if k != "vehicleDetails"}}
                detail['direction'] = direction
                details.append(detail)
    df_details = pd.DataFrame(details)
    return df_details

In [ ]:
#| hide
#| eval: false

df_route_details = extract_route_details(sample_route_details)
print(df_route_details.shape)
df_route_details.head(3)

(238, 34)


,vehicleid,vehiclenumber,servicetypeid,servicetype,centerlat,centerlong,eta,sch_arrivaltime,sch_departuretime,actual_arrivaltime,actual_departuretime,sch_tripstarttime,sch_tripendtime,lastlocationid,currentlocationid,nextlocationid,currentstop,nextstop,laststop,stopCoveredStatus,heading,lastrefreshon,lastreceiveddatetimeflag,tripposition,routeid,stationid,stationname,from,to,routeno,distance_on_station,responsecode,isnotify,direction
0,3441,KA53F0268,72,Non AC/Ordinary,12.97751,77.57141,,15:20,15:20,16:29,16:29,15:20,15:20,0,35088,0,None,None,None,1,0,04-10-2025 18:16:10,0,1,3796,20921,Kempegowda Bus Station,Kempegowda Bus Station,Uttarahalli Bus Stand,210-N,0.00000,200,0,up
1,15201,KA57F3577,72,Non AC/Ordinary,12.97751,77.57141,,17:10,17:10,,,17:10,17:10,0,20939,0,None,None,None,1,174,04-10-2025 18:15:55,0,1,3796,20921,Kempegowda Bus Station,Kempegowda Bus Station,Uttarahalli Bus Stand,210-N,0.00000,200,0,up
2,3441,KA53F0268,72,Non AC/Ordinary,12.97703,77.58580,,15:26,15:26,16:34,16:35,15:20,15:20,0,35088,0,None,None,None,1,0,04-10-2025 18:16:10,0,1,3796,20981,Maharani College,Kempegowda Bus Station,Uttarahalli Bus Stand,210-N,2.08329,200,0,up


In [ ]:
# | export

def extract_map_data(route_details):
    map_data = []
    directions = ['up', 'down']
    for direction in directions:
        vehicles = route_details[direction]['mapData']
        map_data += vehicles
    df_map_data = pd.DataFrame(map_data)
    return df_map_data

In [ ]:
#| hide
#| eval: false

sample_map_data = extract_map_data(sample_route_details)
print(sample_map_data.shape)
sample_map_data.head(3)

(9, 24)


,vehicleid,vehiclenumber,servicetypeid,servicetype,centerlat,centerlong,eta,sch_arrivaltime,sch_departuretime,actual_arrivaltime,actual_departuretime,sch_tripstarttime,sch_tripendtime,lastlocationid,currentlocationid,nextlocationid,currentstop,nextstop,laststop,stopCoveredStatus,heading,lastrefreshon,lastreceiveddatetimeflag,tripposition
0,3441,KA53F0268,72,Non AC/Ordinary,12.909246,77.536713,2025-10-04 18:22:00,16:18,16:18,,,15:20,15:20,0,35088,0,None,None,None,0,0,04-10-2025 18:16:10,0,1
1,15201,KA57F3577,72,Non AC/Ordinary,12.970680,77.586952,2025-10-04 18:54:00,18:08,18:08,,,17:10,17:10,0,20939,0,None,None,None,0,174,04-10-2025 18:15:55,0,1
2,27210,KA57F5807,72,Non AC/Ordinary,12.950208,77.567622,2025-10-04 18:28:00,18:09,18:09,,,17:15,17:15,0,21722,0,None,None,None,0,358,04-10-2025 18:15:12,0,1


In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()